Federated Learning with a Trusted Aggregation

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0805 11:37:29.790584 21900 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\User\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0805 11:37:29.821502 21900 deprecation_wrapper.py:119] From C:\Users\User\Anaconda3\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
#Creating a couple of workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [3]:
data = th.tensor([[1.,1], [0,1], [1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [4]:
bobs_data = data[0:2].send(bob)
bobs_target = data[0:2].send(bob)

In [5]:
alices_data = data[2:].send(alice)
alices_target = data[2:].send(alice)

Now that we divided the data and taregts between bob and target, we need to make a toy model

In [6]:
model = nn.Linear(2,1)

Let;s send this model to bob and alice

In [7]:
bob_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

Optimize the model parameters using an optimizer

In [8]:
bobs_opt = optim.SGD(params=bob_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [9]:
bobs_opt.zero_grad()

In [10]:
bobs_pred = bob_model(bobs_data)
bobs_pred

(Wrapper)>[PointerTensor | me:99779550234 -> bob:58788776281]

In [11]:
bobs_lss = ((bobs_pred - bobs_target)**2).sum()

In [12]:
bobs_lss.backward()

(Wrapper)>[PointerTensor | me:20968526170 -> bob:76949030911]

In [13]:
bobs_opt.step()

In [14]:
bobs_lss = bobs_lss.get().data
bobs_lss

tensor(3.3557)

In [15]:
bobs_opt.zero_grad()
bobs_pred = bob_model(bobs_data)
bobs_lss = ((bobs_pred - bobs_target)**2).sum()
bobs_lss.backward()
bobs_opt.step()
bobs_lss = bobs_lss.get().data
bobs_lss

tensor(2.4418)

In [16]:
alices_opt.zero_grad()
alices_pred = alices_model(alices_data)
alices_lss = ((alices_pred - alices_target)**2).sum()
alices_lss.backward()
alices_opt.step()
alices_lss = alices_lss.get().data
alices_lss

tensor(1.0066)

In [17]:
for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bob_model(bobs_data)
    bobs_lss = ((bobs_pred - bobs_target)**2).sum()
    bobs_lss.backward()
    bobs_opt.step()
    bobs_lss = bobs_lss.get().data
    bobs_lss
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_lss = ((alices_pred - alices_target)**2).sum()
    alices_lss.backward()
    alices_opt.step()
    alices_lss = alices_lss.get().data
    alices_lss

As we have bobs and alices model, we need to average them. To do that we have to send them to the secure worker first. And then take average in that secure worker machine

In [18]:
alices_model.move(secure_worker)
bob_model.move(secure_worker)

In [19]:
secure_worker._objects

{90839851664: Parameter containing:
 tensor([[ 0.4368, -0.6670]], requires_grad=True),
 82810971602: Parameter containing:
 tensor([0.0391], requires_grad=True),
 75156503461: Parameter containing:
 tensor([[ 0.4746, -0.3486]], requires_grad=True),
 33703702079: Parameter containing:
 tensor([0.7787], requires_grad=True)}

Now we are ready to average them. 

In [23]:
with th.no_grad():
    model.weight.set_(((alices_model.weight.data + bob_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bob_model.bias.data) / 2).get())

In [26]:
for round_iter in range(10):
    bob_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    
    bobs_opt = optim.SGD(params=bob_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bob_model(bobs_data)
        bobs_lss = ((bobs_pred - bobs_target)**2).sum()
        bobs_lss.backward()
        bobs_opt.step()
        bobs_lss = bobs_lss.get().data
        bobs_lss

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_lss = ((alices_pred - alices_target)**2).sum()
        alices_lss.backward()
        alices_opt.step()
        alices_lss = alices_lss.get().data
        alices_lss
        
    alices_model.move(secure_worker)
    bob_model.move(secure_worker)
    
    with th.no_grad():
        model.weight.set_(((alices_model.weight.data + bob_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bob_model.bias.data) / 2).get())
    
    #clear the secure_worker 
    secure_worker.clear_objects()
    print("Bob: " + str(bobs_lss) + " Alice:" + str(alices_lss))

Bob: tensor(0.5023) Alice:tensor(0.5001)
Bob: tensor(0.5017) Alice:tensor(0.5001)
Bob: tensor(0.5013) Alice:tensor(0.5001)
Bob: tensor(0.5010) Alice:tensor(0.5000)
Bob: tensor(0.5007) Alice:tensor(0.5000)
Bob: tensor(0.5006) Alice:tensor(0.5000)
Bob: tensor(0.5004) Alice:tensor(0.5000)
Bob: tensor(0.5003) Alice:tensor(0.5000)
Bob: tensor(0.5002) Alice:tensor(0.5000)
Bob: tensor(0.5002) Alice:tensor(0.5000)


Ths kind of set-up is most popular to ensure privacy. 

In [27]:
#Encrypt and Decrypt method

In [28]:
import random

In [29]:
Q = 23740629843760239486723

In [30]:
def encrypt(x, n_shares=3):
    shares = list()
    
    for i in range(n_shares - 1):
        shares.append(random.randint(0,Q))
        
    final_share = Q - (sum(shares) % Q) + x
    shares.append(final_share)
    return tuple(shares)

In [31]:
def decrypt(shares):
    return sum(shares) % Q

In [32]:
decrypt(encrypt(5))

5

In [33]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [34]:
decrypt(add(encrypt(5), encrypt(10)))

15

In [35]:
x = encrypt(5)
x

(9893643993580943702000, 10049885663578952517437, 3797100186600343267291)

In [36]:
y = encrypt(10)
y

(8113761755550289933758, 3608970396934909398154, 12017897691275040154821)

In [37]:
z = add(x, y)
z

(18007405749131233635758, 13658856060513861915591, 15814997877875383422112)

In [38]:
decrypt(z)

15

In [39]:
#Intro to Fixed precision encoding

We need to set aside the space that how much storage we will keel for the numbera that is after the decimal point

In [40]:
base = 10
precision=4

That means its not binary numbers it's base 10. And precision 4 means we are going to allow 4 places after the decimal.

In [41]:
def encode(x_dec):
    return int(x_dec * (base ** precision)) % Q

In [42]:
encode(0.5)

5000

In [47]:
def decode(x_fp):
    return (x_fp if x_fp <= Q/2 else x_fp - Q) / base**precision

In [48]:
decode(5000)

0.5

In [53]:
#Secret Sharing + Fixed Precision in PySyft

In [54]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [55]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [56]:
x = x.share(bob, alice, secure_worker)

This share function will split x into multiple different additive secret shares and send those shares to bob, alice and secure_worker such that we will have pointers to that data. Let's check

In [57]:
bob._objects

{92246702665: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 47104268561: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 58788776281: tensor([[ 0.0365],
         [-0.2067]], grad_fn=<AddmmBackward>), 49416153971: tensor([[1.0081],
         [0.5050]], grad_fn=<AddmmBackward>), 5911119493: tensor([1522676258536665147, 2728966869422825219, 2570556243971290050,
         2852744670096688948,  788391657991178772])}

In [60]:
y = x+x

It creates another set of tensor in bob, alice and secret_worker and y is another additively shared tensor

In [61]:
bob._objects

{92246702665: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 47104268561: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 58788776281: tensor([[ 0.0365],
         [-0.2067]], grad_fn=<AddmmBackward>), 49416153971: tensor([[1.0081],
         [0.5050]], grad_fn=<AddmmBackward>), 5911119493: tensor([1522676258536665147, 2728966869422825219, 2570556243971290050,
         2852744670096688948,  788391657991178772]), 66108951989: tensor([3045352517073330294, 5457933738845650438, 5141112487942580100,
         5705489340193377896, 1576783315982357544])}

In [62]:
y.get()

tensor([ 2,  4,  6,  8, 10])

This .get() decrypts this encrypted tensor

In [64]:
x = th.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [65]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [66]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [69]:
x = th.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [71]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [72]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([ 200,  400,  600,  800, 1000])

In [73]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000, 1.0000])

In [74]:
x = th.tensor([0.1,0.2,0.3]).fix_prec().share(bob, alice, secure_worker)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:14274727669 -> bob:5886695869]
	-> (Wrapper)>[PointerTensor | me:50422529310 -> alice:95231166369]
	-> (Wrapper)>[PointerTensor | me:97135906367 -> secure_worker:36256738096]
	*crypto provider: me*

In [75]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:51192972952 -> bob:96804386406]
	-> (Wrapper)>[PointerTensor | me:75173612379 -> alice:13840665797]
	-> (Wrapper)>[PointerTensor | me:9683429664 -> secure_worker:92983379209]
	*crypto provider: me*

In [76]:
y = y.get().float_prec()
y

tensor([0.2000, 0.4000, 0.6000])